In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", 
    model="Rostlab/prot_bert",
    tokenizer="Rostlab/prot_bert"
)

In [ ]:
mask_filler("MAT[MASK]ALM")

[{'score': 0.9997259974479675, 'token': 25, 'token_str': 'X', 'sequence': 'X'},
 {'score': 4.359659942565486e-05,
  'token': 13,
  'token_str': 'R',
  'sequence': 'R'},
 {'score': 4.001598426839337e-05,
  'token': 5,
  'token_str': 'L',
  'sequence': 'L'},
 {'score': 2.6597530450089835e-05,
  'token': 18,
  'token_str': 'Q',
  'sequence': 'Q'},
 {'score': 2.3989094188436866e-05,
  'token': 10,
  'token_str': 'S',
  'sequence': 'S'}]

# Variant Mutation Processing

In [ ]:
import pandas as pd
import math
from statistics import mean

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
mutations = ['D614G', 'D950N', 'E484K', 'H655Y', 'H655Y', 'K417N', 'N501Y', 'P681H', 'Q493R']

# Common Variant Mutations Masking Testing

In [ ]:
mutations_seqs = {}
[mutations_seqs.setdefault(name, []) for name in mutations]

mutations_seqs_results = {}
[mutations_seqs_results.setdefault(name, []) for name in mutations]

mutations_seqs_original = {}
[mutations_seqs_original.setdefault(name, []) for name in mutations]

In [ ]:
line_count = 0

with open('validation-set.txt', 'r') as file:
    for line in file:
        line_count += 1
        print(line_count)
        for mutation in mutations:
            original = mutation[0]
            change = mutation[-1]
            position = int("".join(mutation[1:-1]))-1

            if position < len(line) and line[position] == original:
                mutations_seqs[mutation].append(line)
                sequence = line[:position] + '[MASK]' + line[position+1:]
                curr_results = mask_filler(sequence)

                curr_sequence_pred = 0
                curr_sequence_nochange = 0

                for i in range(0, len(curr_results)):
                    if curr_results[i]['token_str'] == change:
                        curr_sequence_pred = curr_results[i]['score']
                    elif curr_results[i]['token_str'] == original:
                        curr_sequence_nochange = curr_results[i]['score']
                    if curr_sequence_pred > 0 and curr_sequence_nochange > 0:
                        break
                    
                mutations_seqs_results[mutation].append(curr_sequence_pred)
                mutations_seqs_original[mutation].append(curr_sequence_nochange)

In [ ]:
mutations_seqs_results_mean = {}
[mutations_seqs_results_mean.setdefault(name, 0) for name in mutations]

mutations_seqs_original_mean = {}
[mutations_seqs_original_mean.setdefault(name, 0) for name in mutations]

for mutation in mutations_seqs_original:
    curr_list = mutations_seqs_original[mutation]
    mutations_seqs_original_mean[mutation] = round(mean(curr_list), 2)

for mutation in mutations_seqs_results:
    curr_list = mutations_seqs_results[mutation]
    mutations_seqs_results_mean[mutation] = round(mean(curr_list), 2)

In [ ]:
mutations_seqs_results_mean

{'D614G': 0,
 'D950N': 0,
 'E484K': 0,
 'H655Y': 0,
 'K417N': 0,
 'N501Y': 0,
 'P681H': 0,
 'Q493R': 0.0}

In [ ]:
mutations_seqs_original_mean

{'D614G': 0,
 'D950N': 0,
 'E484K': 0,
 'H655Y': 0,
 'K417N': 0,
 'N501Y': 0,
 'P681H': 0,
 'Q493R': 0.0}

In [ ]:
for mutation in mutations_seqs_results:
    print(len(mutations_seqs_results[mutation]))

68
2063
1799
3776
2012
1878
1708
2171
